In [1]:
import pandas as pd
import re
import os

/Users/i308931/keralarescue/outcall/outcall/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/i308931/keralarescue/outcall/outcall/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
# Regex patterns which are used later
# Mobile phone numbers in India start with 6,7,8 or 9 and has ten digits
# Kerala land lines start with 04 or 4
# Please validate the regex. Improvements are welcome.
mobile_phone_pattern = re.compile("^[6-9]{1}[0-9]{9}$")
lan_phone_pattern = re.compile("^[04]{1}[0-9]{9}$") # TODO: Update regex to filter out numbers which starts with 0 but is not followed immediately by 4

# The directory to use for fetching data and storing results
input_dir = 'input'
results_dir = 'results'

In [3]:
# Method returns dataframe after dropping rows with duplication in specified column
# We'll be using Contact (phone number) as the column to consider while dropping duplicates
def drop_duplicates(df, column):
    return df.drop_duplicates(subset=[column])

# Calculates the number of duplicate rows in a given dataframe
def calculate_n_duplicates(df, column):
    df_without_duplicates = drop_duplicates(df, column)
    n_with_duplicates = df.shape[0] - df_without_duplicates.shape[0]
    return n_with_duplicates

# Prints total number of rows, duplicate rows, and unique rows in the dataframe
# duplicate rows + unique rows = total number of rows
def calc_duplicate_stats(df, column):
    n_total = df.shape[0]
    n_duplicate = calculate_n_duplicates(df, column)
    
    print("Total number of requests: " + str(n_total))
    print("Duplicates in requests: " + str(n_duplicate))
    print("Unique requests: " + str(n_total-n_duplicate))

In [4]:
# returns true is regex matches
def does_regex_match(pattern, string):
    match = pattern.match(string)
    if match is None:
        return False
    else:
        return True

# return the string with all the characters in characters removed
def strip_all_characters_in_list(string, characters):
    for character in characters:
        string = "".join(string.split(character))
    return string

# Removes spaces, minuses, pluses, and dots from the given string and returns it
def strip_number_string(string):
    return strip_all_characters_in_list(string, [" ", "-", ".", "+"])
        
def strip_all_nonnumeric_characters(string):
    new_string = ''.join(ch for ch in string if ch.isdigit())
    return new_string
    
# returns true or false depending on whether or not the given string matches the
# regex for mobile/lan
def is_valid_phone_number(string, is_mobile=True):
    if is_mobile:
        return does_regex_match(mobile_phone_pattern, string)
    else:
        return does_regex_match(lan_phone_pattern, string)

In [5]:
# Returns all the rows with valid Contact
def get_valid_rows(df):
    valid_mobile_rows = []
    valid_lan_rows = []
    invalid_rows = []

    for index, row in df.iterrows():
        contact_phone = row['Contact']
        #print(contact_phone)
        if is_valid_phone_number(strip_all_nonnumeric_characters(contact_phone)): # if valid mobile number
            valid_mobile_rows.append(row)
        elif is_valid_phone_number(strip_all_nonnumeric_characters(contact_phone), is_mobile=False): # if valid lan number
            valid_lan_rows.append(row)
        else:
            invalid_rows.append(row)

    print("# total rows: " + str(df.shape[0]))
    print("# valid mobile rows: " + str(len(valid_mobile_rows)))
    print("# valid lan rows: " + str(len(valid_lan_rows)))
    print("# total valid rows: " + str(len(valid_mobile_rows) + len(valid_lan_rows)))
    print("# invalid rows: " + str(len(invalid_rows)))

    return valid_mobile_rows, valid_lan_rows

In [6]:
# Removes duplicates from the dataframe by Contact column
# Gets the valid phone numbers from it and writes it to separate csvs
# for mobile numbers and landline numbers

def save_csv_for_requests(df, filename_prefix):
    df_without_duplicates = drop_duplicates(df, 'Contact')
    valid_mobile_rows, valid_lan_rows = get_valid_rows(df_without_duplicates)

    labels = list(df)
    
    df_valid_mobile = pd.DataFrame.from_records(valid_mobile_rows, columns=labels)
    file_path = os.path.join(results_dir, filename_prefix + '_valid_mobile_numbers.csv')
    df_valid_mobile.to_csv(file_path, columns=['Contact', 'Person']) # remove the columns parameter to get all the columns
    
    df_valid_lan = pd.DataFrame.from_records(valid_lan_rows, columns=labels)
    file_path = os.path.join(results_dir, filename_prefix + '_valid_lan_numbers.csv')
    df_valid_lan.to_csv(file_path, columns=['Contact', 'Person'])

In [7]:
def cast_columns_to_type_string(df, columns):
    for column in columns:
        df[column] = df[column].astype(str)
    return df

def remove_non_numeric_characters_from_columns(df, columns):
    for column in columns:
        remove_non_numeric = lambda x: strip_all_nonnumeric_characters(x)
        df[column].apply(remove_non_numeric)
    return df

END OF FUNCTION DEFINITIONS

In [8]:
# Reading the excel files

# data_file is the larger dataset, which contains all the rows
data_file = 'data.xlsx'
data_file_path = os.path.join(input_dir, data_file)
df = pd.read_excel(data_file_path, sheet_name='Sheet1')

# subset_file is the smaller data that we want to filter out from the larger dataset
subset_file = 'data_subset.xlsx'
subset_data_file_path = os.path.join(input_dir, subset_file)
df_subset = pd.read_excel(subset_data_file_path, sheet_name='Sheet1')

In [9]:
# Explicitly cast phone number columns to string type
df = cast_columns_to_type_string(df, ['Contact'])
df_subset = cast_columns_to_type_string(df_subset, ['Contact'])

# Strip the phone number column of all non numeric characters
df = remove_non_numeric_characters_from_columns(df, ['Contact'])
df_subset = remove_non_numeric_characters_from_columns(df_subset, ['Contact'])

In [10]:
# list of phone numbers in the subset dataframe
subset_phone_numbers = df_subset['Contact'].values

# apply the filter on the master dataframe
df = df[~df['Contact'].isin(subset_phone_numbers)]

# df.to_csv('filtered', columns=['Contact'])
df.shape

(5, 3)

In [11]:
# Seperating requests made by self and on behalf of others to different dataframes
df_self_request = df.loc[df['Self Request'] == True]
df_behalf_request = df.loc[df['Self Request'] == False]

In [13]:
# Saving the filtered

print("$$ SELF REQUESTS $$")
save_csv_for_requests(df_self_request, "SELF")

print("")

print("$$ BEHALF REQUESTS $$")
save_csv_for_requests(df_behalf_request, "BEHALF")

$$ SELF REQUESTS $$
# total rows: 2
# valid mobile rows: 1
# valid lan rows: 0
# total valid rows: 1
# invalid rows: 1

$$ BEHALF REQUESTS $$
# total rows: 2
# valid mobile rows: 1
# valid lan rows: 1
# total valid rows: 2
# invalid rows: 0
